In [1]:
# 라이브러리 불러오기
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
def video_item(soup, i):
    
    """개별 영상 타입 데이터 리스트 구성 함수"""
    
    # 태그 조회
    t0 = soup.find_all('ytd-video-renderer', {'class':'style-scope ytd-item-section-renderer'})
    t1 = t0[i]
    t2 = t1.find('h3', {'class':'title-and-badge style-scope ytd-video-renderer'})
    t3 = t1.find_all('span', {'class':'style-scope ytd-video-meta-block'})
    
    # 데이터 할당
    channel = t1.find('ytd-channel-name', {'id':'channel-name'}).find('a').text
    title = t2.find('a').text.replace('\n', '')
    url = 'https://www.youtube.com/' + t1.find('ytd-channel-name', {'id':'channel-name'}).find('a')['href']
    views = t3[0].text.replace('조회수', '').replace('회', '').strip()
    date = t3[1].text.replace('전', '').strip()
    
    # 리스트 구성
    video = [title, views, date, channel, url]
    
    return video

In [3]:
def video_playlist(soup, j):
    
    """재생목록 타입 데이터 리스트 구성 함수"""
    
    # 태그 조회
    p0 = soup.find_all('div', {'class':'style-scope ytd-playlist-renderer'})
    p1 = p0[j]
    p2 = p1.find('span', {'id':'video-title'})

    # 데이터 조회
    channel_l = p1.find('div', {'id':'text-container'}).text.replace('\n', '')
    title_l = p2.text.replace('\n', '').strip()
    url_l = 'https://www.youtube.com/' + p1.find('yt-formatted-string', {'id':'view-more'}).find('a')['href']

    # 데이터 리스트 구성
    playlist = [title_l, channel_l, url_l]
    
    return playlist

In [4]:
# 드라이버 실행
url = 'https://www.youtube.com/'
driver = Chrome('C:/Users/toto/Google Drive/sync/python/chromedriver')
driver.get(url)
driver.implicitly_wait(5)

In [5]:
# 검색 및 페이지소스 조회
elem = driver.find_element_by_name('search_query')
elem.send_keys('파이썬 데이터분석')
driver.find_element_by_id('search-icon-legacy').click()
driver.implicitly_wait(5)

body = driver.find_element_by_tag_name('body')

pagedowns = 50
while pagedowns:
    body.send_keys(Keys.END)
    time.sleep(0.5)
    pagedowns -= 1

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [6]:
# 수집 리스트 구성
video_total = []
playlist_total = []

# 리스트 요소 개수
video_len = len(soup.find_all('ytd-video-renderer', {'class':'style-scope ytd-item-section-renderer'}))
playlist_len = len(soup.find_all('div', {'class':'style-scope ytd-playlist-renderer'}))
print('video count = ', video_len)
print('playlist count = ', playlist_len)
print('-'*40)

# 개별 영상 수집
for i in range(video_len):
    video = video_item(soup, i)
    video_total.append(video)
    if i % 50 == 0:
        print('video gathering', i)
print('** video data gathering complete **')
print('-'*40)

# 재생목록 수집
for j in range(playlist_len):
    playlist = video_playlist(soup, j)
    playlist_total.append(playlist)
    if j % 5 == 0:
        print('playlist gathering', j)
print('** playlist data gathering complete **')
print('-'*40)

video count =  452
playlist count =  23
----------------------------------------
video gathering 0
video gathering 50
video gathering 100
video gathering 150
video gathering 200
video gathering 250
video gathering 300
video gathering 350
video gathering 400
video gathering 450
** video data gathering complete **
----------------------------------------
playlist gathering 0
playlist gathering 5
playlist gathering 10
playlist gathering 15
playlist gathering 20
** playlist data gathering complete **
----------------------------------------


In [7]:
# df 구성
video_columns = ['title', 'views', 'date', 'channel', 'url']
playlist_columns = ['title_l', 'channel_l', 'url_l']

video_df = pd.DataFrame(video_total, columns=video_columns)
playlist_df = pd.DataFrame(playlist_total, columns=playlist_columns)
print('video_df shape = ', video_df.shape)
print('playlist_df shape = ', playlist_df.shape)

video_df shape =  (452, 5)
playlist_df shape =  (23, 3)


In [8]:
video_df.head()

,title,views,date,channel,url
0,"파이썬 데이터 분석 - 설치부터 라이브러리 예제까지 (feat. 넘파이, 맷플롯립,...",2.8만,7개월,GongbroDesk,https://www.youtube.com//channel/UCo82I9tH_YvJ...
1,[1/29] 파이썬 데이터 분석을 위한 Pandas 기초 - 판다스 10분 완성소개...,3.2만,2년,todaycode오늘코드,https://www.youtube.com//channel/UCLR3sD0KB_dW...
2,"[1/5] 파이썬 데이터 분석 - 전국 도시 공원 표준 데이터 불러오기, 요약하기",1.3만,2년,todaycode오늘코드,https://www.youtube.com//channel/UCLR3sD0KB_dW...
3,파이썬 판다스로 회사 엑셀데이터 전처리 하는 방법 ! 1탄 (Python / Pan...,5.7천,5개월,데이터 스테이션,https://www.youtube.com//channel/UCyc4xbsDXkgz...
4,Python Pandas 실무 데이터 분석 1탄 (Series / Dataframe...,7.5천,1년,데이터 스테이션,https://www.youtube.com//channel/UCyc4xbsDXkgz...


In [9]:
playlist_df.head()

,title_l,channel_l,url_l
0,이것이 데이터 분석이다 with 파이썬,한빛미디어,https://www.youtube.com//playlist?list=PLVsNiz...
1,[Python] 데이터분석을 위한 파이썬,메모밍 채널 ( Channel Memoming ),https://www.youtube.com//playlist?list=PL-QFFQ...
2,왕초보들을 위한 데이터분석 강의! 모두의 데이터분석 with 파이썬,리송스쿨,https://www.youtube.com//playlist?list=PLZOZh8...
3,Python 데이터 분석과 이미지 처리,동빈나,https://www.youtube.com//playlist?list=PLRx0vP...
4,파이썬 공공데이터 분석,todaycode오늘코드,https://www.youtube.com//playlist?list=PLaTc2c...
